# UAS Akal Imitasi Sentiment Analysis
---
Marsello Ormanda

240310220056

# Import Library
---
Menggunakan library BeautifulSoup dan Transformer dari hugingface

In [ ]:
!pip install requests beautifulsoup4 pandas transformers textblob wordcloud --quiet
print("Instalasi library yang dibutuhkan telah selesai.")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
from transformers import pipeline
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import pandas as pd

# Konfigurasi
---

In [ ]:
NPM = "240310220056"
CSV_FILE_NAME = f"{NPM}_Limabelas.csv"
WORDCLOUD_FILE_NAME = f"{NPM}_WordCloud.png"

# Soal 1: Mengambil Daftar URL Berita dari Halaman Utama
---

## Mengakses url web sumber data scraping
---

In [ ]:
print(">>> BAGIAN 1: Mengambil daftar URL berita...")
base_url = "https://lite.cnn.com"
urls_to_scrape = []
try:
    # Menggunakan headers untuk menyamar sebagai browser agar tidak diblokir
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(base_url, headers=headers)
    response.raise_for_status() # Cek error

    #Parsing website lite cnn
    soup = BeautifulSoup(response.text, 'html.parser')
    all_articles = soup.find_all('li')

    if len(all_articles) >= 52:
        # Mengambil daftar berita sesuai soal
        selected_articles = all_articles[:5] + all_articles[47:52] + all_articles[-5:]
        for article in selected_articles:
            link_tag = article.find('a')
            if link_tag and 'href' in link_tag.attrs:
                full_url = base_url + link_tag['href']
                if full_url not in urls_to_scrape:
                    urls_to_scrape.append(full_url)
        print(f"Selesai. Ditemukan {len(urls_to_scrape)} URL berita unik.")
    else:
        print(f"Peringatan: Hanya ditemukan {len(all_articles)} berita, tidak cukup untuk memenuhi syarat.")

except Exception as e:
    print(f"Gagal mengakses halaman utama CNN Lite. Error: {e}")

# Soal 2 : Mengambil Data Detail dari setiap URL Beritanya
---

In [ ]:
print("\n>>> BAGIAN 2: Mengambil detail dari setiap halaman berita...")
all_news_data = []
if urls_to_scrape:
    # membuat loop untuk iterasi setiap url yang ada di list
    for i, url in enumerate(urls_to_scrape, 1):
        try:
            # parsing dari url yang ada
            page = requests.get(url, headers=headers)
            soup_detail = BeautifulSoup(page.text, 'html.parser')

            # Mengambil Judul
            judul_tag = soup_detail.find("div", class_="headline--lite")
            judul = judul_tag.find("h2", class_="headline").text.strip() if judul_tag else "N/A"

            # Mengambil Penulis
            penulis_tag = soup_detail.find('p', class_="byline--lite")
            penulis = penulis_tag.text.replace("By", "").strip() if penulis_tag else "N/A"

            # KOREKSI 2: Logika baru untuk mengambil Tanggal dan Jam
            tanggal = "N/A"
            jam = "N/A"
            timestamp_tag = soup_detail.find('p', class_="timestamp--lite")
            if timestamp_tag:
                full_timestamp_text = timestamp_tag.text.strip()

                # Mengambil semua teks di antara ':' pertama dan ',' pertama
                if ":" in full_timestamp_text and "," in full_timestamp_text:
                    start = full_timestamp_text.find(':') + 1
                    end = full_timestamp_text.find(',')
                    jam = full_timestamp_text[start:end].strip()

                # Mengambil Tanggal: bagian setelah koma pertama
                if "," in full_timestamp_text:
                    tanggal_parts = full_timestamp_text.split(',')[1:]
                    tanggal = ','.join(tanggal_parts).strip()

            # Mengambil sumber berita
            sumber_tag = soup_detail.find('p', class_="source--lite")
            sumber = sumber_tag.text.strip() if sumber_tag else "N/A"

            # Mengambil Baris Pertama Berita
            baris_pertama_tag = soup_detail.find('p', class_='paragraph--lite')
            baris_pertama = baris_pertama_tag.text.strip() if baris_pertama_tag else "N/A"

            # Mengambil Link Utuh
            link_lengkap = "N/A"
            # Mencari tag <a> yang teksnya mengandung frasa tertentu
            link_lengkap_tag = soup_detail.find('a', string=re.compile(r"See Full Web Article"))
            if link_lengkap_tag and link_lengkap_tag.has_attr('href'):
                link_lengkap = link_lengkap_tag['href']

            # Menambahkan semua data yang sudah diekstrak dengan benar ke list
            all_news_data.append({
                "Judul": judul,
                "Penulis": penulis,
                "Tanggal Terbit": tanggal,
                "Jam Terbit": jam,
                "Sumber Berita": sumber,
                "Baris Pertama Berita": baris_pertama,
                "Link Berita Utuh": link_lengkap,
                "Link Berita Lite": url
            })
            print(f"  ({i}/{len(urls_to_scrape)}) Berhasil scrape: {judul}")

        except Exception as e:
            print(f"  ({i}/{len(urls_to_scrape)}) Gagal scrape {url}. Error: {e}")
else:
    print("Tidak ada URL untuk di-scrape.")

## Cek Hasil Scraping Data Detail Berita
---

In [ ]:
all_news_data

In [ ]:
df_news = pd.DataFrame(all_news_data)

print("\nDataFrame Data Detail Berita:")
print(df_news)

print(f"\nDataFrame memiliki {len(df_news)} baris dan {len(df_news.columns)} kolom.")

In [ ]:
df_news.isnull()

---
Terdeteksi tidak ada data yang null atau kosong, sehingga semua detail berita berhasil ter-scrape atau terambil

In [ ]:
df_news.to_csv('240310220056_Limabelas(without sentiment).csv', index=False)
print("DataFrame df_news telah disimpan ke dalam file CSV dengan nama 240310220056_Limabelas(wihout sentiment).csv")

# Soal 3 : Analisis Sentimen dengan Text Blob dan juga Model Huggingface
---

## Analisis Sentiment text_blob
---

In [ ]:
polarities = []
subjectivities = []

In [ ]:
for teks in df_news["Baris Pertama Berita"]:
  result = TextBlob(teks)

  polarity = result.sentiment.polarity
  subjectivity = result.sentiment.subjectivity

  polarities.append(polarity)
  subjectivities.append(subjectivity)

In [ ]:
df_news["polarity_textblob"] = polarities
df_news["subjectivity_textblob"] = subjectivities

In [ ]:
df_news.head()

## Analisis Sentimen dengan Huggingface
---

In [ ]:
model = "ProsusAI/finbert"
analyze = pipeline("text-classification", model=model)

In [ ]:
analyze(df_news["Baris Pertama Berita"][0])

In [ ]:
labels = []
scores = []

for teks in df_news["Baris Pertama Berita"]:
  result = analyze(teks)
  HF_label = result[0]["label"]
  HF_score = result[0]["score"]

  labels.append(HF_label)
  scores.append(HF_score)

In [ ]:
df_news["HFLabel"] = labels
df_news["HFScores"] = scores

In [ ]:
df_news.head()

In [ ]:
df_news.to_csv('240310220056_Limabelas(with sentiment).csv', index=False)
print("DataFrame df_news telah disimpan ke dalam file CSV dengan nama 240310220056_Limabelas(with sentiment).csv")

# Soal 4 :  WordCloud dari Baris Pertama Berita
---

In [ ]:
# Gabung semua teks dari kolom Baris Pertama Berita
all_text = " ".join(df_news['Baris Pertama Berita'].dropna())

In [ ]:
# Buat WordCloud dengan ukuran 16:9
wordcloud = WordCloud(width=1600, height=900, background_color='white').generate(all_text)

In [ ]:
# Tampilkan wordcloud
plt.figure(figsize=(16, 9))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off') # Jangan tampilkan sumbu
plt.title('Word Cloud dari Baris Pertama Berita')

In [ ]:
# Simpan wordcloud ke file
plt.savefig(WORDCLOUD_FILE_NAME, dpi=300, bbox_inches='tight')

plt.show()
print(f"WordCloud berhasil dibuat dan disimpan sebagai {WORDCLOUD_FILE_NAME}")

In [ ]:
from google.colab import files

df_news.to_csv(CSV_FILE_NAME, index=False)

print(f"\nDataFrame berhasil disimpan ke dalam file CSV: {CSV_FILE_NAME}")

files.download(CSV_FILE_NAME)
print(f"File CSV '{CSV_FILE_NAME}' siap untuk diunduh.")